# Redes Complexas:  Lista02

**Aluno:** Gustavo Siqueira Barbosa

**Nº USP:** 10728122

**Docente:** Francisco Aparecido Rodrigues, francisco@icmc.usp.br.

Universidade de São Paulo, São Carlos, Brasil.


In [2]:
# Importando bibliotecas utilizadas
import networkx as nx
import numpy as np
import math

### Questão 01
Para a rede social Hamsterster, calcule a média dos menores caminhos e o diâmetro. Considere apenas o maior componente.

In [11]:
import matplotlib.pyplot as plt

# Lendo os dados
G= nx.read_edgelist("data/hamsterster.txt", nodetype=int, data=(('weight',float),))
G = G.to_undirected()
G.remove_edges_from(nx.selfloop_edges(G))
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
G = nx.convert_node_labels_to_integers(G, first_label=0)

# Calculando a média dos menores caminhos
avg_shortest_path = nx.average_shortest_path_length(G)

# Calculando o diâmetro (caminho mais curto com a maior medida)
diameter = nx.diameter(G)

print("Média dos menores caminhos:%.3f"%avg_shortest_path)
print("Diâmetro:%.3f"%diameter)

Média dos menores caminhos:3.453
Diâmetro:14.000


### Questão 02
Para a rede de aeroportos (base USairport500), calcule a média e variância do comprimento dos menores caminhos. Considere apenas o maior componente na rede.

In [40]:
# Lendo os dados
G= nx.read_edgelist("data/USairport500.txt", nodetype=int, data=(('weight',float),))
G = G.to_undirected()
G.remove_edges_from(nx.selfloop_edges(G))
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
G = nx.convert_node_labels_to_integers(G, first_label=0)

# Calculando a média dos menores caminhos
avg_shortest_path = nx.average_shortest_path_length(G)

# Armazenando os menores caminhos em um vetor
N = G.number_of_nodes()
v = np.zeros( int((N**2 - N)/2) )
v_idx = 0
for i in range(N):
    for j in range(i+1, N):
        aux = nx.shortest_path(G,i,j)
        v[v_idx] = len(aux)-1
        v_idx += 1

# Calculando a variância
var = np.var(v)
print("Média dos menores caminhos: %.3f"%avg_shortest_path)
print("Variância: %.3f"%np.var(v))

Média dos menores caminhos: 2.991
Variância: 0.818


### Questão 03
Calcule o coeficiente de assortatividade para a rede Advogato. Considere apenas o maior componente.

In [5]:
# Lendo os dados
G= nx.read_edgelist("data/advogato.txt", nodetype=int, data=(('weight',float),))
G = G.to_undirected()
G.remove_edges_from(nx.selfloop_edges(G))
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
G = nx.convert_node_labels_to_integers(G, first_label=0)

# Calculando o coeficiente de assortatividade
coef_assortativity = nx.degree_assortativity_coefficient(G)
print("Assortatividade: %3f"%coef_assortativity)

Assortatividade: -0.095721


### Questão 04
Para a rede de aeroportos (USairport500), calcule a entropia de Shannon considerando o comprimento dos menores caminhos. Use o logaritmo na base 2 e considere apenas o maior componente.

In [42]:
# Lendo os dados
G= nx.read_edgelist("data/USairport500.txt", nodetype=int, data=(('weight',float),))
G = G.to_undirected()
G.remove_edges_from(nx.selfloop_edges(G))
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
G = nx.convert_node_labels_to_integers(G, first_label=0)

# Função para o cálculo da entropia de Shannon
def shannon_entropy(deg_arr):
    
    #deg_arr = np.array(list( (dict(G.degree())).values() ))
    P = np.zeros(np.max(deg_arr)+1) 
    
    for k in deg_arr:
        P[k] += 1
    P /= sum(P) 
     
    H = 0
    for pk in P:
        if pk > 0: H -= pk * math.log(pk,2)
    return H

# Armazenando os menores caminhos em um vetor
N = G.number_of_nodes()
v = np.zeros( int((N**2 - N)/2) )
v_idx = 0
for i in range(N):
    for j in range(i+1, N):
        aux = nx.shortest_path(G,i,j)
        v[v_idx] = len(aux)-1
        v_idx += 1
        
print(shannon_entropy(v.astype(int)))

1.883667007854659


### Questão 05
Calcule o coeficiente de correlação de Pearson entre o grau médio dos vizinhos e o grau para a rede de palavras (word_adjacencies). Considere apenas o maior componente conectado.

In [69]:
# Lendo os dados
G= nx.read_edgelist("data/word_adjacencies.txt", nodetype=int, data=(('weight',float),))
G = G.to_undirected()
G.remove_edges_from(nx.selfloop_edges(G))
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
G = nx.convert_node_labels_to_integers(G, first_label=0)


N = G.number_of_nodes()

knn = np.zeros(N)
k_idx = 0
for i in G.nodes():
    aux =  nx.average_neighbor_degree(G, nodes = [i])
    knn[k_idx] = float(aux[i])
    k_idx += 1

vk = list(dict(G.degree()).values())

vk_min = np.min(vk)
vk_max = np.max(vk)

knnk = np.zeros(vk_max - vk_min + 1)
ks = np.zeros(vk_max - vk_min + 1)

ks_idx = 0
for k in np.arange(vk_min, vk_max+1):
    aux = vk == k
    if(len(knn[aux]) > 0):
        avg_knn = np.mean(knn[aux]) 
        
        knnk[ks_idx] = avg_knn
        ks[ks_idx] = k
        
        ks_idx += 1

knnk = knnk[:ks_idx]
ks = ks[:ks_idx]

coef_pearson = np.corrcoef(ks, knnk)[0,1]
print('Coeficiente de correlação de Pearson: %.3f'%coef_pearson)

Coeficiente de correlação de Pearson: -0.711
